# Carregando Ambiente

In [1]:
// Importando as bibliotecas necessárias
const faker = require('faker');
const mongoose = require('mongoose');
mongoose.set('useCreateIndex', true);
const _ = require('lodash');
const countryData = require('country-data');
const moment = require('moment');

// Conectando a base de dados usando mongoose
mongoose.connect('mongodb://localhost/effective_store', { useUnifiedTopology: true, useNewUrlParser: true }).then(() => console.log('Connected'));
console.log('Connecting...')

Connecting...
Connected


# Subindo Dados e Schemas na Base

In [2]:
// Definindo os países disponíveis
const availableCountries = _.filter(countryData.countries.all, countrie => countrie.status === 'assigned');

// Definindo o status de pagamento disponível
const availableStatus = ['Payed', 'Waiting Payment', 'Canceled']

// Definindo schema de produto
const ProductSchema = new mongoose.Schema({
  name: String,
  description: String,
  price: Number
});

// Definindo o schema de Invoice
const InvoiceSchema = new mongoose.Schema({
  invoiceCode: {
    type: String,
    unique: true
  },
  dueDate: Date,
  status: {
    type: String,
    enum: availableStatus
  },
  productList: [ProductSchema],
  totalAmount: Number,
  invoiceDoc: {
    sellerCode: {
      type: String,
      unique: true
    },
    docCode: { 
      type: String,
      unique: true
    }
  }
})

// Definindo o model de Invoice
const Invoice = mongoose.model('Invoice', InvoiceSchema);

// Definindo o schema de Sellers
const SellerSchema = new mongoose.Schema({
  code: {
    type: String,
    unique: true
  },
  name: String,
  country: {
    code: { type: String, enum: availableCountries.map(country => country.alpha2) },
    name: { type: String, enum: availableCountries.map(country => country.name) },
    emojiFlag: { type: String, enum: availableCountries.map(country => country.emoji) }
  },
  invoices: [InvoiceSchema],
  createdAt: Date,
  productCatalog: [ProductSchema]
});

// Definindo o model de Seller
const Seller = mongoose.model('Seller', SellerSchema);


console.log('Done!');

Done!


# Gerando dados de sellers

In [3]:
const SELLERS_LENGTH = 500;
const PRODUCTS_LENGTH = 100;

const availableProducts = _.range(PRODUCTS_LENGTH).map(() => ({
  name: faker.commerce.product(),
  price: parseFloat(faker.commerce.price()),
  description: faker.commerce.productDescription(),
}));

const sellers = _.range(SELLERS_LENGTH).map((sellerIndex) => {
  const productCatalogSize = _.random(1, PRODUCTS_LENGTH)
  const country = _.sample(availableCountries)
  const productCatalog = []
  _.range(productCatalogSize).forEach(() => {
    const productSample = _.sample(availableProducts)
    if (!productCatalog.includes(productSample)) {
      productCatalog.push(productSample)
    }
  });
  
  return new Seller({
    code: _.uniqueId(`seller_cod_${sellerIndex}_`),
    name: faker.company.companyName(),
    country: { code: country.alpha2, name: country.name, emojiFlag: country.emoji },
    createdAt: faker.date.past(10),
    productCatalog
  });
});
console.log('Done!');

Done!


# Gerando dados de invoices

In [4]:
const MAX_SALES_BY_A_SELLER = 5

const salesNumbers = _.range(MAX_SALES_BY_A_SELLER)
const setSellerInvoice = seller => {
  const invoices = []
  _.range(_.sample(salesNumbers)).forEach((index) => {
    const productCatalogRange = _.range(seller.productCatalog.length)
    const invoiceProductsLength = _.sample(productCatalogRange)
    const productList = _.range(invoiceProductsLength).map(() => _.sample(seller.productCatalog))
    if (!_.isEmpty(productList)) {
      invoices.push(new Invoice({
        invoiceCode: _.uniqueId(`invoice_${seller.code}`),
        dueDate: faker.date.past(10),
        status: _.sample(availableStatus),
        productList,
        totalAmount: _.reduce(_.map(productList, product => product.price), (price, memo) => price + memo , 0),
        invoiceDoc: {
          sellerCode: seller.code,
          docCode: _.uniqueId(`doc_${seller.code}`)
        }
      }))
    }
  })

  seller.invoices = invoices
}

sellers.forEach(setSellerInvoice)
console.log('Done!');

Done!


# Populando base de dados
Limpamos a collection de sellers e após isto inserimos os dados gerados
## Atenção aguardar o log de Inserted

In [5]:
console.log('Cleaning sellers collection')
Seller.deleteMany({}, () => {
  console.log('cleared')
  console.log('Inserting generated data')
  Seller.insertMany(sellers, () => console.log('Inserted') )
})

console.log('Processing...');

Cleaning sellers collection
Processing...
cleared
Inserting generated data
Inserted


# Query 1 - Lojistas (Sellers) com maior número de produtos disponíveis na plataforma

In [6]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      productCatalogSize: { $cond: { if: { $isArray: "$productCatalog" }, then: { $size: "$productCatalog" }, else: "NA"} }
    }
  },
  {
    $sort : { productCatalogSize : -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...');

Processing...
┌─────────┬─────────────────────┬────────────────────┐
│ (index) │        name         │ productCatalogSize │
├─────────┼─────────────────────┼────────────────────┤
│    0    │   'Koss - Dooley'   │         68         │
│    1    │   'Crooks Group'    │         68         │
│    2    │     'Rohan Inc'     │         67         │
│    3    │   'Borer - Mills'   │         67         │
│    4    │ 'Beahan - MacGyver' │         66         │
│    5    │ 'Lebsack - Walter'  │         65         │
│    6    │  'Greenfelder LLC'  │         65         │
│    7    │ 'Leannon - Labadie' │         65         │
│    8    │ 'Ritchie and Sons'  │         64         │
│    9    │  'Nolan - Conroy'   │         64         │
└─────────┴─────────────────────┴────────────────────┘


# Query 2 - Produtos mais comuns entre os lojistas associados

In [7]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...');

Processing...
┌─────────┬────────────┬────────┐
│ (index) │    _id     │ counts │
├─────────┼────────────┼────────┤
│    0    │  'Shoes'   │  1351  │
│    1    │  'Pizza'   │  1335  │
│    2    │  'Pants'   │  1143  │
│    3    │   'Bike'   │  1121  │
│    4    │  'Salad'   │  1114  │
│    5    │  'Table'   │  949   │
│    6    │   'Car'    │  949   │
│    7    │   'Ball'   │  944   │
│    8    │ 'Chicken'  │  944   │
│    9    │   'Soap'   │  929   │
│   10    │  'Bacon'   │  760   │
│   11    │ 'Computer' │  759   │
│   12    │  'Cheese'  │  756   │
│   13    │  'Chair'   │  754   │
│   14    │  'Gloves'  │  748   │
│   15    │ 'Sausages' │  730   │
│   16    │ 'Keyboard' │  592   │
│   17    │   'Tuna'   │  568   │
│   18    │  'Chips'   │  564   │
│   19    │  'Mouse'   │  544   │
│   20    │   'Hat'    │  418   │
│   21    │   'Fish'   │  387   │
│   22    │  'Towels'  │  378   │
│   23    │  'Shirt'   │  201   │
└─────────┴────────────┴────────┘


# Query 3 - Países com mais lojistas

In [8]:
Seller.aggregate([
  {
    $group: {
      _id: '$country.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...');

Processing...
┌─────────┬─────────────────────────────────────┬────────┐
│ (index) │                 _id                 │ counts │
├─────────┼─────────────────────────────────────┼────────┤
│    0    │               'Peru'                │   7    │
│    1    │             'Botswana'              │   6    │
│    2    │            'Switzerland'            │   6    │
│    3    │             'Greenland'             │   5    │
│    4    │             'Slovakia'              │   5    │
│    5    │              'Mayotte'              │   5    │
│    6    │              'Guinea'               │   5    │
│    7    │               'Chad'                │   5    │
│    8    │              'Germany'              │   5    │
│    9    │ 'Heard Island And McDonald Islands' │   5    │
└─────────┴─────────────────────────────────────┴────────┘


# Query 4 - Produtos com mais lojistas por região

In [9]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      country: { $first : "$country.name" },
      counts: { $sum: 1 },
    }
  },
  {
    $sort: { country: -1 }
  }
], (errors, result) => console.table(result));
console.log('Processing...');

Processing...
┌─────────┬────────────┬──────────────┬────────┐
│ (index) │    _id     │   country    │ counts │
├─────────┼────────────┼──────────────┼────────┤
│    0    │  'Shirt'   │ 'Sri Lanka'  │  201   │
│    1    │  'Towels'  │ 'Mauritania' │  378   │
│    2    │   'Fish'   │  'Iceland'   │  387   │
│    3    │   'Bike'   │    'Chad'    │  1121  │
│    4    │  'Bacon'   │    'Chad'    │  760   │
│    5    │  'Chips'   │    'Chad'    │  564   │
│    6    │  'Pants'   │    'Chad'    │  1143  │
│    7    │   'Car'    │    'Chad'    │  949   │
│    8    │   'Tuna'   │    'Chad'    │  568   │
│    9    │ 'Sausages' │    'Chad'    │  730   │
│   10    │  'Table'   │    'Chad'    │  949   │
│   11    │  'Mouse'   │    'Chad'    │  544   │
│   12    │ 'Keyboard' │    'Chad'    │  592   │
│   13    │  'Gloves'  │    'Chad'    │  748   │
│   14    │  'Cheese'  │    'Chad'    │  756   │
│   15    │   'Ball'   │    'Chad'    │  944   │
│   16    │  'Chair'   │    'Chad'    │  754   │
│   17

# Query 5 - Lista dos lojistas (Sellers) ordenada pelos mais novos

In [10]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result));
console.log('Processing...');

Processing...
┌─────────┬───────────────────────────────────┬──────────────────────────┐
│ (index) │               name                │        createdAt         │
├─────────┼───────────────────────────────────┼──────────────────────────┤
│    0    │        'Stamm - Lindgren'         │ 2020-09-06T21:34:05.827Z │
│    1    │ 'Ruecker, Christiansen and Ferry' │ 2020-09-04T16:39:34.254Z │
│    2    │          'Marks - Kozey'          │ 2020-08-12T14:26:47.080Z │
│    3    │         'Kuhn - Lockman'          │ 2020-08-09T04:36:10.953Z │
│    4    │            'Grant Inc'            │ 2020-08-02T00:44:30.241Z │
│    5    │          'Jenkins - Orn'          │ 2020-07-27T15:18:43.292Z │
│    6    │         'Emard - Carroll'         │ 2020-07-24T22:03:59.335Z │
│    7    │           'Mueller LLC'           │ 2020-07-23T14:39:45.380Z │
│    8    │      'Konopelski - Schultz'       │ 2020-07-16T05:14:59.681Z │
│    9    │        'Waelchi - Windler'        │ 2020-07-13T18:42:30.492Z │
└─────────┴

# Query 6 - Lista dos lojistas (Sellers) ordenada pelos mais antiga

In [11]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: 1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result));
console.log('Processing...');

Processing...
┌─────────┬────────────────────────────────┬──────────────────────────┐
│ (index) │              name              │        createdAt         │
├─────────┼────────────────────────────────┼──────────────────────────┤
│    0    │         'Hartmann LLC'         │ 2010-09-19T14:37:31.497Z │
│    1    │       'Pfannerstill Inc'       │ 2010-09-26T23:49:59.898Z │
│    2    │         'Crona Group'          │ 2010-10-07T05:27:46.321Z │
│    3    │       'Hauck - Kerluke'        │ 2010-10-12T01:47:22.860Z │
│    4    │       'Herman and Sons'        │ 2010-10-12T13:16:17.875Z │
│    5    │ 'Nienow, Romaguera and Barton' │ 2010-10-14T12:19:53.203Z │
│    6    │         'Lang - Marks'         │ 2010-10-15T04:41:08.383Z │
│    7    │          'Mohr Group'          │ 2010-10-23T13:19:20.753Z │
│    8    │        'Robel - Kuphal'        │ 2010-10-24T13:58:57.548Z │
│    9    │      'Donnelly - Spencer'      │ 2010-10-25T06:57:23.135Z │
└─────────┴────────────────────────────────┴──────

# Query 7 - Lista das faturas que estão em aberto para os lojistas(Sellers) com menos de 1 ano de mercado

In [23]:
Seller.aggregate([
  {
    $unwind: '$invoices'
  },
  {
    $match: { $and: [{ 'invoices.status': 'Waiting Payment' }, { createdAt: { $gt: new Date(moment().subtract('year', 1).format()) } }] }
  },
  {
    $project: {
        '_id': 0,
        'name': 1,
        ''
        'invoices.status': 1,
        'invoices.totalAmount': 1
    }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...');

Processing...
┌─────────┬────────────────────────┬───────────────────────────────────────────────────┐
│ (index) │          name          │                     invoices                      │
├─────────┼────────────────────────┼───────────────────────────────────────────────────┤
│    0    │     'Mueller LLC'      │  { status: 'Waiting Payment', totalAmount: 923 }  │
│    1    │   'Greenfelder LLC'    │ { status: 'Waiting Payment', totalAmount: 4930 }  │
│    2    │   'Greenfelder LLC'    │ { status: 'Waiting Payment', totalAmount: 10727 } │
│    3    │    'Rogahn - Ferry'    │ { status: 'Waiting Payment', totalAmount: 4099 }  │
│    4    │    'Rogahn - Ferry'    │ { status: 'Waiting Payment', totalAmount: 5990 }  │
│    5    │     'Kunde - Yost'     │ { status: 'Waiting Payment', totalAmount: 10513 } │
│    6    │    'Bode and Sons'     │ { status: 'Waiting Payment', totalAmount: 11039 } │
│    7    │    'Bode and Sons'     │ { status: 'Waiting Payment', totalAmount: 2951 }  │
│    8 

# Query 8 - Lista das faturas acima de 1000,00

In [ ]:
Seller.aggregate([
  {
    $unwind: '$invoices'
  },
  {
    $match: { 'invoices.totalAmount': { $gt: 1000 } }
  },
  {
    $sort: { 'invoices.totalAmout': 1 }
  },
  {
    $project: {
        '_id': 0,
        'invoices.invoiceCode': 1,
        'invoices.totalAmount': 1
    }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...');